In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## L

In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
np.random.seed(42)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
### Pretrained Word Embeddings

folder_path = "/content/drive/MyDrive/model_imp_emo_6_run_4/"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
CPU times: user 1.42 s, sys: 405 ms, total: 1.82 s
Wall time: 3.58 s


## BECODEMIXED 2x Dataset kiiiiiiiiii

In [ ]:
# %%time
# becodemixed_data =  pd.read_excel('/content/drive/MyDrive/model_imp_whole_2/be_data_no_un_combined.xlsx') 


In [ ]:
%%time
train_data =  pd.read_excel('/content/drive/MyDrive/model_imp_emo_6_run_4/train.xlsx') 
valid_data = pd.read_excel('/content/drive/MyDrive/model_imp_emo_6_run_4/val.xlsx')
test_data = pd.read_excel('/content/drive/MyDrive/model_imp_emo_6_run_4/test.xlsx')

CPU times: user 424 ms, sys: 22.5 ms, total: 446 ms
Wall time: 1.3 s


In [ ]:
# import random  

# def split_three_ways(dataset, ratio1, ratio2, ratio3):
#   # shuffle dataset
  
  
#   cp_dataset = dataset.sample(frac=1)
#   total = len(dataset)

#   d = ratio1 + ratio2 + ratio3
#   size_train = round(round(ratio1 / d, 2) * total)

#   size_rest = total - size_train

#   size_valid = round(round(ratio2 / (ratio2 + ratio3), 2) * size_rest)
#   size_test = size_rest - size_valid

#   train_data = cp_dataset[0:size_train]
#   valid_data = cp_dataset[size_train : size_train + size_valid]
#   test_data = cp_dataset[size_train + size_valid:]
#   return train_data, valid_data, test_data 

# train_data, valid_data, test_data = split_three_ways(becodemixed_data, 80, 10, 10)
# print(len(train_data), len(valid_data), len(test_data))


In [ ]:
valid_data.head(2)

,Label,Text
0,Disgust,"To be honest the movie kinda bothers me , just..."
1,Joy,I finnaly got to see a osadharon movie onekdin...


In [ ]:
train_data['Label'].value_counts()

Joy         1363
Sadness      826
Disgust      771
Anger        515
Fear         349
Surprise     229
Name: Label, dtype: int64

In [ ]:
train_data['enc_label'] = train_data['Label'].replace({'Joy': 0,'Disgust':  3,'Anger' :4,
                                                       'Sadness': 1,'Surprise':2,'Fear': 5})
valid_data['enc_label'] = valid_data['Label'].replace({'Joy': 0,'Disgust':  3,'Anger' :4,
                                                       'Sadness': 1,'Surprise':2,'Fear': 5})

In [ ]:
train_data.head(1)

,Label,Text,enc_label
0,Joy,Ami matroi gaan ta abar shunlam 😊 Eta # CHAAM...,0


In [ ]:
test_data.head(1)

,Unnamed: 0,Text
0,0,BD te to mukh dekhe bichar hoy...hafez alem ma...


In [ ]:
# train_data['Text'].tolist()

In [ ]:
train_data['enc_label'].value_counts()

0    1363
1     826
3     771
4     515
5     349
2     229
Name: enc_label, dtype: int64

In [ ]:
valid_data['enc_label'].value_counts()

0    157
1    118
3     90
4     59
5     44
2     39
Name: enc_label, dtype: int64

In [ ]:
valid_data['Label'].value_counts()

Joy         157
Sadness     118
Disgust      90
Anger        59
Fear         44
Surprise     39
Name: Label, dtype: int64

In [ ]:
print("Size of the training data ==> ", train_data.shape)
print("Size of the validation data ==>", valid_data.shape)
print("Size of the Test data ==>", test_data.shape)

Size of the training data ==>  (4053, 3)
Size of the validation data ==> (507, 3)
Size of the Test data ==> (507, 2)


## One Hot Encoding

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoder=OneHotEncoder(sparse=False)

encoded_labels = pd.DataFrame (encoder.fit_transform(train_data[['Label']]))

encoded_labels .columns = encoder.get_feature_names(['Label'])

train_data= pd.concat([train_data, encoded_labels ], axis=1)

In [ ]:
train_data = train_data.rename(columns={'Label_Joy': 'Joy',
                                        'Label_Disgust':  'Disgust','Label_Anger' :'Anger',
                                        'Label_Sadness': 'Sadness',
                                        'Label_Surprise':'Surprise','Label_Fear': 'Fear'
                                      
                                      })
train_data.head(1)

,Label,Text,enc_label,Anger,Disgust,Fear,Joy,Sadness,Surprise
0,Joy,Ami matroi gaan ta abar shunlam 😊 Eta # CHAAM...,0,0.0,0.0,0.0,1.0,0.0,0.0


## Cleaning

In [ ]:
'''
Text Cleaning
'''
def text_cleaning(row):
  # to remove HTML tags
  text = BeautifulSoup(row, 'html.parser').get_text()
  d = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE) #This line is for removing url
  post = d.replace('\n', '')
  post = post.replace('—', ' ')
  # to remove accented characters
  new_text = unicodedata.normalize('NFKD', post).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  # to remove special characters and numbers
  # define the pattern to keep
  #pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
  #text = re.sub(pat, '', new_text)
  #to remove punctuation
  text = ''.join([c for c in text if c not in string.punctuation])
  # to remove special characters
  pattern = r'^\s*|\s\s*'
  text = re.sub(pattern, ' ', text).strip()
  # convert into lower case
  text = text.lower() 
  # Stopword Removing
  tokenizer = ToktokTokenizer()
  # convert sentence into token of words
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  # check in lowercase 
  #t = [token for token in tokens if token.lower() not in stopword_list]
  #text = ' '.join(t) 
  # Stemming
  #t = [stemming.stem(token) for token in tokens ]
  #text = ' '.join(t) 

  return text



In [ ]:
def get_becodemixed_stop_words():
    #swdf1 = pd.read_csv("/content/drive/MyDrive/automated_tamil_text_classification-master/data/TamilNLP_TamilStopWords.txt",  header=None) 
    swdf2 = pd.read_csv("/content/drive/MyDrive/model_imp_emo_6_run_4/custom_becodemixed_stopwords.txt",  header=None) 
    #sw1 = swdf1[0].tolist()
    sw2 = swdf2[0].tolist()
    #tamil_stop_words = list(set(sw1 + sw2))
    becodemixed_stop_words = list(sw2)
    return becodemixed_stop_words

In [ ]:
#removing URLs and Punctuations
#train_data['cleaned'] = train_data['tweet'].apply(remove_url)
#Removing punctuations


becodemixed_stop_words = get_becodemixed_stop_words()
valid_data['Texts'] = valid_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))
train_data['Texts'] = train_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))
test_data['Texts'] = test_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))

train_data['cleaned'] = train_data['Texts'].apply(text_cleaning)
valid_data['cleaned'] = valid_data['Texts'].apply(text_cleaning)
test_data['cleaned'] = test_data['Texts'].apply(text_cleaning)


## Data samples after cleaning
print("Original Data:===\n",train_data.Text[0],"\nCleaned Data:===\n",train_data.cleaned[0],)

Original Data:===
  Ami matroi gaan ta abar shunlam 😊 Eta # CHAAMP er best song.. Arijit 's voice is love & video tar jonn…  
Cleaned Data:===
 ami matroi gaan shunlam 😊 eta chaamp best song arijit s voice is love video tar jonn…


In [ ]:
valid_data['cleaned'].shape

(507,)

##ML

In [ ]:
def print_metrices(pred,true):
    print(confusion_matrix(true,pred))
    print(classification_report(true,pred,))
    print("Accuracy : ",accuracy_score(pred,true))
    print("Precison : ",precision_score(pred,true, average = 'weighted'))
    print("Recall : ",recall_score(pred,true,  average = 'weighted'))
    print("F1 : ",f1_score(pred,true,  average = 'weighted'))

In [ ]:
## TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

tfidfvectorizer = TfidfVectorizer(max_features=40000,ngram_range=(1,2),tokenizer=lambda x: x.split())
tfidfvectorised = tfidfvectorizer.fit_transform(train_data['cleaned'])
tfidfvectorized_train = tfidfvectorizer.transform(train_data['cleaned'])
tfidfvectorized_valid = tfidfvectorizer.transform(valid_data['cleaned'])
tfidfvectorized_test = tfidfvectorizer.transform(test_data['cleaned'])

print("Feature Size: ", tfidfvectorized_train.shape[1])


Feature Size:  40000


In [ ]:
tfidfvectorized_valid.shape

(507, 40000)

In [ ]:
valid_data['Label'].value_counts()

Joy         157
Sadness     118
Disgust      90
Anger        59
Fear         44
Surprise     39
Name: Label, dtype: int64

In [ ]:

actual_labels = pd.read_excel('/content/drive/MyDrive/model_imp_emo_6_run_4/test_with_label_forwhole.xlsx')

actual_labels 

,label,text
0,Anger,BD te to mukh dekhe bichar hoy...hafez alem ma...
1,Joy,Bette Davis er obhonoy in the lead role really...
2,Sadness,yeta kintu 100% sotti anyway ki hoyeche tmr e...
3,Sadness,bhalo lagche na kichui .  feeling alone
4,Fear,IT coding just kk? Kal just kono bachar upae d...
...,...,...
502,Disgust,Akta thirty minutes er story ke stretch korey ...
503,Joy,Amsterdam theke ki shilpo anlen ? Okhane to e...
504,Sadness,Asa ses ... tv off 😖 😖 r ki6u korar nei gelo h...
505,Joy,Amra dev ke e biswas kri apnake na . Apni...


In [ ]:
labels = ['Joy','Sadness','Surprise','Disgust','Anger','Fear']
def actual_label(row):
    return labels[row]

### Logistic Regression

In [ ]:
lr = LogisticRegression(class_weight='balanced',C =10)
lr.fit(tfidfvectorized_train,train_data['enc_label'])
y_pred = lr.predict(tfidfvectorized_valid)
print_metrices(y_pred,valid_data['enc_label'])

[[114  19   5  13   5   1]
 [ 32  59   2  18   6   1]
 [ 17   8   3   9   2   0]
 [  8  14   1  54  13   0]
 [  3  11   0   8  36   1]
 [  2   2   0   3   2  35]]
              precision    recall  f1-score   support

           0       0.65      0.73      0.68       157
           1       0.52      0.50      0.51       118
           2       0.27      0.08      0.12        39
           3       0.51      0.60      0.55        90
           4       0.56      0.61      0.59        59
           5       0.92      0.80      0.85        44

    accuracy                           0.59       507
   macro avg       0.57      0.55      0.55       507
weighted avg       0.58      0.59      0.58       507

Accuracy :  0.5936883629191322
Precison :  0.6260758429487234
Recall :  0.5936883629191322
F1 :  0.6067134781177209


In [ ]:
test_pred = lr.predict(tfidfvectorized_test).tolist()
test_data['id'] = test_data.index + 1

In [ ]:
df = pd.DataFrame(test_data['id'])
df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,text,label
0,1,BD te to mukh dekhe bichar hoy...hafez alem ma...,Anger
1,2,Bette Davis er obhonoy in the lead role really...,Joy
2,3,yeta kintu 100% sotti anyway ki hoyeche tmr e...,Joy
3,4,bhalo lagche na kichui .  feeling alone,Sadness
4,5,IT coding just kk? Kal just kono bachar upae d...,Sadness
...,...,...,...
502,503,Akta thirty minutes er story ke stretch korey ...,Disgust
503,504,Amsterdam theke ki shilpo anlen ? Okhane to e...,Surprise
504,505,Asa ses ... tv off 😖 😖 r ki6u korar nei gelo h...,Sadness
505,506,Amra dev ke e biswas kri apnake na . Apni...,Disgust


In [ ]:
df['label'].value_counts()

Joy         207
Disgust      91
Sadness      90
Anger        61
Fear         38
Surprise     20
Name: label, dtype: int64

In [ ]:
df.to_csv(folder_path +'becodemixed_logisticRegression.tsv',sep = '\t',index = False)

In [ ]:
print_metrices(actual_labels['label'],df['label'])

[[ 31  11   4   8   6   1]
 [  5  48   1  16  17   4]
 [  0   1  37   0   0   0]
 [ 10  20   5 142  16  14]
 [ 10  14   4  14  46   2]
 [  0   3   0   7   1   9]]
              precision    recall  f1-score   support

       Anger       0.55      0.51      0.53        61
     Disgust       0.49      0.53      0.51        91
        Fear       0.73      0.97      0.83        38
         Joy       0.76      0.69      0.72       207
     Sadness       0.53      0.51      0.52        90
    Surprise       0.30      0.45      0.36        20

    accuracy                           0.62       507
   macro avg       0.56      0.61      0.58       507
weighted avg       0.63      0.62      0.62       507

Accuracy :  0.6173570019723866
Precison :  0.6213362822236498
Recall :  0.6173570019723866
F1 :  0.6156963443352809


## DT

In [ ]:
dt = DecisionTreeClassifier(class_weight='balanced')
dt.fit(tfidfvectorized_train,train_data['enc_label'])
y_pred = dt.predict(tfidfvectorized_valid)
print_metrices(y_pred,valid_data['enc_label'])

[[74 23 13 23 16  8]
 [26 50  6 20 12  4]
 [10  1  5  9 10  4]
 [16 19 10 37  8  0]
 [ 5  7  1 15 30  1]
 [ 5  1  0  6  3 29]]
              precision    recall  f1-score   support

           0       0.54      0.47      0.51       157
           1       0.50      0.42      0.46       118
           2       0.14      0.13      0.14        39
           3       0.34      0.41      0.37        90
           4       0.38      0.51      0.43        59
           5       0.63      0.66      0.64        44

    accuracy                           0.44       507
   macro avg       0.42      0.43      0.42       507
weighted avg       0.45      0.44      0.45       507

Accuracy :  0.4437869822485207
Precison :  0.44792030779196107
Recall :  0.4437869822485207
F1 :  0.44228198709291183


In [ ]:
test_pred = dt.predict(tfidfvectorized_test).tolist()
test_data['id'] = test_data.index + 1

In [ ]:
df = pd.DataFrame(test_data['id'])
df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,text,label
0,1,BD te to mukh dekhe bichar hoy...hafez alem ma...,Anger
1,2,Bette Davis er obhonoy in the lead role really...,Anger
2,3,yeta kintu 100% sotti anyway ki hoyeche tmr e...,Surprise
3,4,bhalo lagche na kichui .  feeling alone,Disgust
4,5,IT coding just kk? Kal just kono bachar upae d...,Disgust
...,...,...,...
502,503,Akta thirty minutes er story ke stretch korey ...,Joy
503,504,Amsterdam theke ki shilpo anlen ? Okhane to e...,Surprise
504,505,Asa ses ... tv off 😖 😖 r ki6u korar nei gelo h...,Joy
505,506,Amra dev ke e biswas kri apnake na . Apni...,Anger


In [ ]:
print_metrices(actual_labels['label'],df['label'])

[[23 21  5 18 11  1]
 [10 26  3 18 16  8]
 [ 2  3 33  7  4  2]
 [ 7 24  5 93 25 14]
 [10 18  4 28 27  4]
 [ 4  5  1 23  3  1]]
              precision    recall  f1-score   support

       Anger       0.41      0.29      0.34        79
     Disgust       0.27      0.32      0.29        81
        Fear       0.65      0.65      0.65        51
         Joy       0.50      0.55      0.52       168
     Sadness       0.31      0.30      0.31        91
    Surprise       0.03      0.03      0.03        37

    accuracy                           0.40       507
   macro avg       0.36      0.36      0.36       507
weighted avg       0.40      0.40      0.40       507

Accuracy :  0.40039447731755423
Precison :  0.41476282328923414
Recall :  0.40039447731755423
F1 :  0.4053823184330418


### SVM

In [ ]:
# C : float, default=1.0
#     Regularization parameter. The strength of the regularization is inversely proportional to C. Must be strictly positive
# The "balanced" mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y)).

In [ ]:
## SVM
from sklearn.svm import SVC
svc =  SVC(C= 6, kernel='rbf' ,random_state=265,class_weight='balanced')
svc.fit(tfidfvectorized_train,train_data['enc_label'])
y_pred = svc.predict(tfidfvectorized_valid)
print_metrices(y_pred,valid_data['enc_label'])

[[133  13   0  11   0   0]
 [ 45  56   0  15   2   0]
 [ 22   5   0  10   2   0]
 [ 24  12   0  50   4   0]
 [ 12  13   0   8  26   0]
 [  9   4   0   2   0  29]]
              precision    recall  f1-score   support

           0       0.54      0.85      0.66       157
           1       0.54      0.47      0.51       118
           2       0.00      0.00      0.00        39
           3       0.52      0.56      0.54        90
           4       0.76      0.44      0.56        59
           5       1.00      0.66      0.79        44

    accuracy                           0.58       507
   macro avg       0.56      0.50      0.51       507
weighted avg       0.56      0.58      0.55       507

Accuracy :  0.5798816568047337
Precison :  0.6782231699748869
Recall :  0.5798816568047337
F1 :  0.6074522183189554


In [ ]:
test_pred = svc.predict(tfidfvectorized_test).tolist()
test_data['id'] = test_data.index + 1

In [ ]:
df = pd.DataFrame(test_data['id'])
df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,text,label
0,1,BD te to mukh dekhe bichar hoy...hafez alem ma...,Joy
1,2,Bette Davis er obhonoy in the lead role really...,Joy
2,3,yeta kintu 100% sotti anyway ki hoyeche tmr e...,Joy
3,4,bhalo lagche na kichui .  feeling alone,Sadness
4,5,IT coding just kk? Kal just kono bachar upae d...,Sadness
...,...,...,...
502,503,Akta thirty minutes er story ke stretch korey ...,Joy
503,504,Amsterdam theke ki shilpo anlen ? Okhane to e...,Surprise
504,505,Asa ses ... tv off 😖 😖 r ki6u korar nei gelo h...,Sadness
505,506,Amra dev ke e biswas kri apnake na . Apni...,Joy


In [ ]:
df['label'].value_counts()

Joy         286
Sadness      80
Disgust      75
Anger        32
Fear         32
Surprise      2
Name: label, dtype: int64

In [ ]:
df.to_csv(folder_path +'becodemixed_svm.tsv',sep = '\t',index = False)

In [ ]:
print_metrices(actual_labels['label'],df['label'])

[[ 22   5   1   2   2   0]
 [  4  44   0  10  10   7]
 [  0   1  31   0   0   0]
 [ 21  37  11 164  32  21]
 [  9  10   8  10  42   1]
 [  0   0   0   1   0   1]]
              precision    recall  f1-score   support

       Anger       0.39      0.69      0.50        32
     Disgust       0.45      0.59      0.51        75
        Fear       0.61      0.97      0.75        32
         Joy       0.88      0.57      0.69       286
     Sadness       0.49      0.53      0.51        80
    Surprise       0.03      0.50      0.06         2

    accuracy                           0.60       507
   macro avg       0.48      0.64      0.50       507
weighted avg       0.70      0.60      0.63       507

Accuracy :  0.5996055226824457
Precison :  0.6157666388509584
Recall :  0.5996055226824457
F1 :  0.5735538134373188


## MNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb =   MultinomialNB(alpha=0.1)#alpha : float, default=1.0 additive (Laplace/Lidstone) smoothing parameter (0 for no smoothing).
nb.fit(tfidfvectorized_train,train_data['enc_label'])
y_pred = nb.predict(tfidfvectorized_valid)
print_metrices(y_pred,valid_data['enc_label'])

[[127  15   0  14   1   0]
 [ 49  53   0  14   1   1]
 [ 23   8   0   6   2   0]
 [ 23  14   0  49   4   0]
 [ 10  11   0   8  29   1]
 [  7   2   0   2   1  32]]
              precision    recall  f1-score   support

           0       0.53      0.81      0.64       157
           1       0.51      0.45      0.48       118
           2       0.00      0.00      0.00        39
           3       0.53      0.54      0.54        90
           4       0.76      0.49      0.60        59
           5       0.94      0.73      0.82        44

    accuracy                           0.57       507
   macro avg       0.55      0.50      0.51       507
weighted avg       0.55      0.57      0.55       507

Accuracy :  0.571992110453649
Precison :  0.6580522568398236
Recall :  0.571992110453649
F1 :  0.5978758556224911


In [ ]:
test_pred = nb.predict(tfidfvectorized_test).tolist()
test_data['id'] = test_data.index

In [ ]:
df = pd.DataFrame(test_data['id'])
# df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,label
0,0,Sadness
1,1,Joy
2,2,Joy
3,3,Sadness
4,4,Joy
...,...,...
502,502,Disgust
503,503,Surprise
504,504,Sadness
505,505,Joy


In [ ]:
df['label'].value_counts()

Joy         269
Sadness      84
Disgust      81
Fear         35
Anger        33
Surprise      5
Name: label, dtype: int64

In [ ]:
df.to_csv(folder_path +'becodemixed_mnbfinal.tsv',sep = '\t',index = False)

In [ ]:
print_metrices(actual_labels['label'],df['label'])

[[ 20   5   2   4   2   0]
 [  6  46   0   7  17   5]
 [  0   1  34   0   0   0]
 [ 16  33  11 163  24  22]
 [ 14  12   3  12  43   0]
 [  0   0   1   1   0   3]]
              precision    recall  f1-score   support

       Anger       0.36      0.61      0.45        33
     Disgust       0.47      0.57      0.52        81
        Fear       0.67      0.97      0.79        35
         Joy       0.87      0.61      0.71       269
     Sadness       0.50      0.51      0.51        84
    Surprise       0.10      0.60      0.17         5

    accuracy                           0.61       507
   macro avg       0.49      0.64      0.52       507
weighted avg       0.69      0.61      0.63       507

Accuracy :  0.6094674556213018
Precison :  0.6191415148267053
Recall :  0.6094674556213018
F1 :  0.5877047103922921


##Deep Learning

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LSTM,GRU
from tensorflow.keras.models import load_model
print("Tensorflow Version:--",tf.version.VERSION)
print("Keras Version:---",keras.__version__)

Tensorflow Version:-- 2.8.2
Keras Version:--- 2.8.0


###Tokenization

In [ ]:
tokenizer = Tokenizer(num_words = 10000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n-', 
                      split=' ', char_level=False, oov_token='<oov>', document_count=0)
tokenizer.fit_on_texts(train_data['Text'])  
vocab_size = len(tokenizer.word_index)+1
vocab_size

12591

## what is thattttttt

1.def def pad_sequences(sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.0)
Pads sequences to the same length.

This function transforms a list (of length num_samples) of sequences (lists of integers) into a 2D Numpy array of shape (num_samples, num_timesteps). num_timesteps is either the maxlen argument if provided, or the length of the longest sequence in the list.
2.Public API for tf.keras.preprocessing.sequence namespace.



In [ ]:
max_len = 300
# Training Sequences
train_sequences = tokenizer.texts_to_sequences(train_data['Text']) #def texts_to_sequences(texts)Transforms each text in texts to a sequence of integers.
print(len(tokenizer.word_index))
train_pad_sequences =  keras.preprocessing.sequence.pad_sequences(train_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Training Sequences :" ,train_pad_sequences.shape)

# Validation Sequences
validation_sequences = tokenizer.texts_to_sequences(valid_data['Text'])
validation_pad_sequences =  keras.preprocessing.sequence.pad_sequences(validation_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Validation Sequences :" ,validation_pad_sequences.shape)

12590
Number of Training Sequences : (4053, 300)
Number of Validation Sequences : (507, 300)


In [ ]:
# Test Sequences
test_sequences = tokenizer.texts_to_sequences(test_data['Text'])
test_pad_sequences =  keras.preprocessing.sequence.pad_sequences(test_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Test Sequences :" ,test_pad_sequences.shape)

Number of Test Sequences : (507, 300)


In [ ]:

# Estimate class weights for unbalanced datasets.

# Parameters

# class_weight : dict, 'balanced' or None
#     If 'balanced', class weights will be given by
# n_samples / (n_classes * np.bincount(y)).
# If a dictionary is given, keys are classes and values
# are corresponding class weights.
# If None is given, the class weights will be uniform.

# classes : ndarray

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced'
                                               ,classes =np.unique(train_data['enc_label'])
                                               ,y=train_data['enc_label'])

weight = {i : class_weights[i] for i in range(6)}

In [ ]:
weight

{0: 0.4955979457079971,
 1: 0.8177966101694916,
 2: 2.949781659388646,
 3: 0.8761348897535668,
 4: 1.311650485436893,
 5: 1.9355300859598854}

In [ ]:
# def def __init__(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch', options=None, **kwargs)
# Callback to save the Keras model or model weights at some frequency.

# ModelCheckpoint callback is used in conjunction with training using model.fit() to save a model or weights (in a checkpoint file) at some interval, so the model or weights can be loaded later to continue the training from the state saved.

# A few options this callback provides include:

# Whether to only keep the model that has achieved the "best performance" so far, or whether to save the model at the end of every epoch regardless of performance.
# Definition of 'best'; which quantity to monitor and whether it should be maximized or minimized.
# The frequency it should save at. Currently, the callback supports saving at the end of every epoch, or after a fixed number of training batches.
# Whether only weights are saved, or the whole model is saved.
# Note: If you get WARNING:tensorflow:Can save best model only with <name> available, skipping see the description of the monitor argument for details on how to get this right.


# def def __init__(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None, **kwargs)
# Turns positive integers (indexes) into dense vectors of fixed size.

# e.g. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]

# This layer can only be used as the first layer in a model.

# Example:

# >>> model = tf.keras.Sequential()
# >>> model.add(tf.keras.layers.Embedding(1000, 64, input_length=10))
# >>> # The model will take as input an integer matrix of size (batch,
# >>> # input_length), and the largest integer (i.e. word index) in the input
# >>> # should be no larger than 999 (vocabulary size).
# >>> # Now model.output_shape is (None, 10, 64), where `None` is the batch
# >>> # dimension.

##cnn

In [ ]:
num_classes = 6

accuracy_threshold = 0.99

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>accuracy_threshold):
        print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
        self.model.stop_training = True

acc_callback = myCallback()
# Saved the Best Model
filepath = folder_path+"becodemixed_cnn.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
# callback list
callback_list = [acc_callback, checkpoint] 
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000, 100, input_length = max_len),
# def def __init__(filters, kernel_size, strides=1, padding='valid', data_format='channels_last', dilation_rate=1, groups=1, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None, **kwargs)
# 1D convolution layer (e.g. temporal convolution).
tf.keras.layers.Conv1D(128, 5, activation='relu'),
tf.keras.layers.MaxPooling1D(pool_size=2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(6 , activation='softmax')])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 100)          1000000   
                                                                 
 conv1d (Conv1D)             (None, 296, 128)          64128     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 148, 128)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 18944)             0         
                                                                 
 dense (Dense)               (None, 6)                 113670    
                                                                 
Total params: 1,177,798
Trainable params: 1,177,798
Non-trainable params: 0
______________________________________________

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.003),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
   
model.fit(train_pad_sequences,train_data['enc_label'], epochs=30, batch_size = 32, validation_split = 0.2,verbose =1,callbacks = callback_list,class_weight = weight)

Epoch 1/30
102/102 [==============================] - ETA: 0s - loss: 1.7048 - accuracy: 0.2258
Epoch 1: val_accuracy improved from -inf to 0.47472, saving model to /content/drive/MyDrive/model_imp_emo_6_run_4/becodemixed_cnn.h5
102/102 [==============================] - 15s 12ms/step - loss: 1.7048 - accuracy: 0.2258 - val_loss: 1.4316 - val_accuracy: 0.4747
Epoch 2/30
100/102 [============================>.] - ETA: 0s - loss: 0.8446 - accuracy: 0.6737
Epoch 2: val_accuracy improved from 0.47472 to 0.49938, saving model to /content/drive/MyDrive/model_imp_emo_6_run_4/becodemixed_cnn.h5
102/102 [==============================] - 1s 7ms/step - loss: 0.8430 - accuracy: 0.6743 - val_loss: 1.3740 - val_accuracy: 0.4994
Epoch 3/30
 92/102 [==========================>...] - ETA: 0s - loss: 0.1531 - accuracy: 0.9504
Epoch 3: val_accuracy improved from 0.49938 to 0.54624, saving model to /content/drive/MyDrive/model_imp_emo_6_run_4/becodemixed_cnn.h5
102/102 [==============================] - 

In [ ]:
# Load the saved model
model = load_model(folder_path+'becodemixed_cnn.h5')
# prediction
y_pred = np.argmax(model.predict(validation_pad_sequences), axis=-1)#numpy.argmax(a, axis=None, out=None, *, keepdims=<no value>)Returns the indices of the maximum values along an axis.
print_metrices(valid_data['enc_label'],y_pred)


[[100  28  17  13   4   5]
 [ 26  60  12  12  17   1]
 [  2   0   1   0   0   1]
 [ 20  17   7  51   5   2]
 [  6  10   2  13  31   3]
 [  3   3   0   1   2  32]]
              precision    recall  f1-score   support

           0       0.64      0.60      0.62       167
           1       0.51      0.47      0.49       128
           2       0.03      0.25      0.05         4
           3       0.57      0.50      0.53       102
           4       0.53      0.48      0.50        65
           5       0.73      0.78      0.75        41

    accuracy                           0.54       507
   macro avg       0.50      0.51      0.49       507
weighted avg       0.58      0.54      0.56       507

Accuracy :  0.5424063116370809
Precison :  0.5257483254457591
Recall :  0.5424063116370809
F1 :  0.5260955051474312


In [ ]:
test_pred = np.argmax(model.predict(test_pad_sequences), axis=-1).tolist()
# test_pred
test_data['id'] = test_data.index + 1
df = pd.DataFrame(test_data['id'])
# df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,label
0,1,Disgust
1,2,Joy
2,3,Joy
3,4,Sadness
4,5,Joy
...,...,...
502,503,Disgust
503,504,Surprise
504,505,Sadness
505,506,Disgust


In [ ]:
df['label'].value_counts()

Joy         191
Disgust     110
Sadness      99
Anger        59
Fear         35
Surprise     13
Name: label, dtype: int64

In [ ]:
print_metrices(actual_labels['label'],df['label'])

[[ 27  14   6   2  10   0]
 [ 10  48   2  25  15  10]
 [  0   1  34   0   0   0]
 [ 12  15   5 132  13  14]
 [  7  18   4  21  46   3]
 [  0   1   0   7   2   3]]
              precision    recall  f1-score   support

       Anger       0.48      0.46      0.47        59
     Disgust       0.49      0.44      0.46       110
        Fear       0.67      0.97      0.79        35
         Joy       0.71      0.69      0.70       191
     Sadness       0.53      0.46      0.50        99
    Surprise       0.10      0.23      0.14        13

    accuracy                           0.57       507
   macro avg       0.50      0.54      0.51       507
weighted avg       0.58      0.57      0.57       507

Accuracy :  0.571992110453649
Precison :  0.5791233204952886
Recall :  0.571992110453649
F1 :  0.5703422672544552


In [ ]:

df.to_csv(folder_path +'becodemixed_cnnfinal.tsv',sep = '\t',index = False)

## lstm

In [ ]:
num_classes = 6
from keras.layers.embeddings import Embedding
accuracy_threshold = 0.99

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>accuracy_threshold):
        print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
        self.model.stop_training = True

acc_callback = myCallback()
# Saved the Best Model
filepath = folder_path+"becodemixed_LSTM.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
# callback list
callback_list = [acc_callback, checkpoint] 
# lstm
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000, 100, input_length = max_len),

tf.keras.layers.LSTM(units = 128,activation='tanh',return_sequences=True,dropout = 0.2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(6 , activation='softmax')])

model.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 lstm_1 (LSTM)               (None, 300, 128)          117248    
                                                                 
 flatten_2 (Flatten)         (None, 38400)             0         
                                                                 
 dense_2 (Dense)             (None, 6)                 230406    
                                                                 
Total params: 1,347,654
Trainable params: 1,347,654
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.003),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
   
model.fit(train_pad_sequences,train_data['enc_label'], epochs=30, batch_size = 32, validation_split = 0.2,verbose =1,callbacks = callback_list,class_weight = weight)

Epoch 1/30
102/102 [==============================] - ETA: 0s - loss: 1.9492 - accuracy: 0.2742
Epoch 1: val_accuracy improved from -inf to 0.42540, saving model to /content/drive/MyDrive/model_imp_emo_6_run_4/becodemixed_LSTM.h5
102/102 [==============================] - 6s 28ms/step - loss: 1.9492 - accuracy: 0.2742 - val_loss: 1.4112 - val_accuracy: 0.4254
Epoch 2/30
100/102 [============================>.] - ETA: 0s - loss: 0.6642 - accuracy: 0.7491
Epoch 2: val_accuracy improved from 0.42540 to 0.51788, saving model to /content/drive/MyDrive/model_imp_emo_6_run_4/becodemixed_LSTM.h5
102/102 [==============================] - 2s 21ms/step - loss: 0.6621 - accuracy: 0.7502 - val_loss: 1.3823 - val_accuracy: 0.5179
Epoch 3/30
102/102 [==============================] - ETA: 0s - loss: 0.1225 - accuracy: 0.9571
Epoch 3: val_accuracy did not improve from 0.51788
102/102 [==============================] - 2s 22ms/step - loss: 0.1225 - accuracy: 0.9571 - val_loss: 1.8719 - val_accuracy: 0

In [ ]:
# Load the saved model
model = load_model(folder_path+'becodemixed_LSTM.h5')
# prediction
y_pred = np.argmax(model.predict(validation_pad_sequences), axis=-1)#numpy.argmax(a, axis=None, out=None, *, keepdims=<no value>)Returns the indices of the maximum values along an axis.
print_metrices(valid_data['enc_label'],y_pred)


[[101  21  10   7   3   2]
 [ 20  61   8  15   9   3]
 [  4   4   1   0   0   1]
 [ 23  25  14  53   2   3]
 [  8   5   6  14  43   2]
 [  1   2   0   1   2  33]]
              precision    recall  f1-score   support

           0       0.64      0.70      0.67       144
           1       0.52      0.53      0.52       116
           2       0.03      0.10      0.04        10
           3       0.59      0.44      0.50       120
           4       0.73      0.55      0.63        78
           5       0.75      0.85      0.80        39

    accuracy                           0.58       507
   macro avg       0.54      0.53      0.53       507
weighted avg       0.61      0.58      0.59       507

Accuracy :  0.5759368836291914
Precison :  0.563266646843978
Recall :  0.5759368836291914
F1 :  0.563961242777266


In [ ]:
test_pred = np.argmax(model.predict(test_pad_sequences), axis=-1).tolist()
# test_pred
test_data['id'] = test_data.index + 1
df = pd.DataFrame(test_data['id'])
# df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,label
0,1,Sadness
1,2,Joy
2,3,Joy
3,4,Sadness
4,5,Joy
...,...,...
502,503,Disgust
503,504,Surprise
504,505,Sadness
505,506,Anger


In [ ]:
df['label'].value_counts()

Joy         186
Disgust     133
Sadness      69
Anger        64
Fear         40
Surprise     15
Name: label, dtype: int64

In [ ]:
print_metrices(actual_labels['label'],df['label'])

[[ 31  10   5   7  11   0]
 [  9  59   3  33  20   9]
 [  0   3  36   0   1   0]
 [  8  15   6 129  12  16]
 [  7  10   1  10  40   1]
 [  1   0   0   8   2   4]]
              precision    recall  f1-score   support

       Anger       0.55      0.48      0.52        64
     Disgust       0.61      0.44      0.51       133
        Fear       0.71      0.90      0.79        40
         Joy       0.69      0.69      0.69       186
     Sadness       0.47      0.58      0.52        69
    Surprise       0.13      0.27      0.18        15

    accuracy                           0.59       507
   macro avg       0.53      0.56      0.53       507
weighted avg       0.61      0.59      0.59       507

Accuracy :  0.5897435897435898
Precison :  0.5988238580647629
Recall :  0.5897435897435898
F1 :  0.5880009342955312


## bilstm

In [ ]:
num_classes = 6

accuracy_threshold = 0.99

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>accuracy_threshold):
        print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
        self.model.stop_training = True

acc_callback = myCallback()
# Saved the Best Model
filepath = folder_path+"becodemixed_BiLSTM.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
# callback list
callback_list = [acc_callback, checkpoint] 
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000, 100, input_length = max_len),
#tf.keras.layers.Conv1D(128, 5, activation='relu'),
#tf.keras.layers.MaxPooling1D(5),
tf.keras.layers.Bidirectional(LSTM(units = 128,return_sequences=True,dropout = 0.2)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(6 , activation='softmax')])

model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 bidirectional (Bidirectiona  (None, 300, 256)         234496    
 l)                                                              
                                                                 
 flatten_3 (Flatten)         (None, 76800)             0         
                                                                 
 dense_3 (Dense)             (None, 6)                 460806    
                                                                 
Total params: 1,695,302
Trainable params: 1,695,302
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.003),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
   
model.fit(train_pad_sequences,train_data['enc_label'], epochs=30, batch_size = 32, validation_split = 0.2,verbose =1,callbacks = callback_list,class_weight = weight)

Epoch 1/30
101/102 [============================>.] - ETA: 0s - loss: 1.6350 - accuracy: 0.3122
Epoch 1: val_accuracy improved from -inf to 0.46856, saving model to /content/drive/MyDrive/model_imp_emo_6_run_4/becodemixed_BiLSTM.h5
102/102 [==============================] - 8s 45ms/step - loss: 1.6342 - accuracy: 0.3131 - val_loss: 1.3670 - val_accuracy: 0.4686
Epoch 2/30
101/102 [============================>.] - ETA: 0s - loss: 0.5636 - accuracy: 0.7843
Epoch 2: val_accuracy improved from 0.46856 to 0.53391, saving model to /content/drive/MyDrive/model_imp_emo_6_run_4/becodemixed_BiLSTM.h5
102/102 [==============================] - 4s 37ms/step - loss: 0.5623 - accuracy: 0.7850 - val_loss: 1.5006 - val_accuracy: 0.5339
Epoch 3/30
101/102 [============================>.] - ETA: 0s - loss: 0.1267 - accuracy: 0.9629
Epoch 3: val_accuracy improved from 0.53391 to 0.55487, saving model to /content/drive/MyDrive/model_imp_emo_6_run_4/becodemixed_BiLSTM.h5
102/102 [=========================

In [ ]:
# Load the saved model
model = load_model(folder_path+'becodemixed_BiLSTM.h5')
# prediction
y_pred = np.argmax(model.predict(validation_pad_sequences), axis=-1)#numpy.argmax(a, axis=None, out=None, *, keepdims=<no value>)Returns the indices of the maximum values along an axis.
print_metrices(valid_data['enc_label'],y_pred)


[[108  36  16  11   5   3]
 [ 15  55   6  21   8   2]
 [  2   2   1   0   0   1]
 [ 21  21   8  46   5   2]
 [  8   3   7  12  39   1]
 [  3   1   1   0   2  35]]
              precision    recall  f1-score   support

           0       0.69      0.60      0.64       179
           1       0.47      0.51      0.49       107
           2       0.03      0.17      0.04         6
           3       0.51      0.45      0.48       103
           4       0.66      0.56      0.60        70
           5       0.80      0.83      0.81        42

    accuracy                           0.56       507
   macro avg       0.52      0.52      0.51       507
weighted avg       0.60      0.56      0.58       507

Accuracy :  0.5601577909270217
Precison :  0.5357253108206975
Recall :  0.5601577909270217
F1 :  0.5418949115344417


### Test Set Prediction

In [ ]:
test_pred = np.argmax(model.predict(test_pad_sequences), axis=-1).tolist()
# test_pred

In [ ]:
test_pred = np.argmax(model.predict(test_pad_sequences), axis=-1).tolist()
# test_pred
test_data['id'] = test_data.index 
df = pd.DataFrame(test_data['id'])
# df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,label
0,0,Sadness
1,1,Joy
2,2,Joy
3,3,Sadness
4,4,Joy
...,...,...
502,502,Disgust
503,503,Surprise
504,504,Sadness
505,505,Anger


In [ ]:
df['label'].value_counts()

Joy         199
Disgust     111
Anger        68
Sadness      64
Fear         44
Surprise     21
Name: label, dtype: int64

In [ ]:
print_metrices(actual_labels['label'],df['label'])

[[ 31  12   6   8  11   0]
 [  8  49   1  28  21   4]
 [  1   4  37   0   1   1]
 [  9  18   6 132  16  18]
 [  6  10   1  11  36   0]
 [  1   4   0   8   1   7]]
              precision    recall  f1-score   support

       Anger       0.55      0.46      0.50        68
     Disgust       0.51      0.44      0.47       111
        Fear       0.73      0.84      0.78        44
         Joy       0.71      0.66      0.68       199
     Sadness       0.42      0.56      0.48        64
    Surprise       0.23      0.33      0.27        21

    accuracy                           0.58       507
   macro avg       0.52      0.55      0.53       507
weighted avg       0.59      0.58      0.58       507

Accuracy :  0.5759368836291914
Precison :  0.5791928919643283
Recall :  0.5759368836291914
F1 :  0.5736487298639814


In [ ]:
df.to_csv(folder_path +'becodemixed_bilstmfinal3.tsv',sep ='\t',index = False)

In [ ]:
DataFrame.drop(labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise')

NameError: ignored

In [ ]:
df.to_csv(folder_path +'becodemixed_bilstmfinal2.tsv',sep = '\t',index = False)

In [ ]:
%%time
t =  pd.read_csv('/content/drive/MyDrive/submit/CUET16_run1xb.tsv')

t1 =  pd.read_csv('/content/drive/MyDrive/submit/CUET16_run2mlb.tsv')
t2 =  pd.read_csv('/content/drive/MyDrive/submit/CUET16_run3BL.tsv')

In [ ]:

t2



# Data Statistics

In [ ]:
df=train_data 
df['word_count'] = df.cleaned.apply(lambda x: len(str(x).split()))
more_than_150 =  df['word_count'] > 150
df3 = df[more_than_150]
df3

,Label,Text,enc_label,Anger,Disgust,Fear,Joy,Sadness,Surprise,Texts,cleaned,word_count


In [ ]:
def char_length(row):
    return len(row.split())

In [ ]:
lables = ['Joy','Sadness','Surprise','Disgust','Anger','Fear']

# Count number of unique words in each set
def dataset_statistics(dataset):
  for l in lables:
    word_list = [word.strip() for t in list(dataset[dataset[l]==1.0].cleaned) for word in t.strip().split()]
    print(f"\nClass Name:--> {l} ") 
    print("Number of Words:{}".format(len(word_list))) 
    print("Number of Unique Words:{}".format(len(np.unique(word_list)))) 
    print("Maximum post length: {}".format(max(dataset[dataset[l]==1.0].cleaned.apply(char_length))))
    print("Average Words in texts: {}".format(np.mean(dataset[dataset[l]==1.0].cleaned.apply(char_length))))
    

In [ ]:
train_data.head(2)

,Label,Text,enc_label,Anger,Disgust,Fear,Joy,Sadness,Surprise,Texts,cleaned,word_count
0,Joy,Ami matroi gaan ta abar shunlam 😊 Eta # CHAAM...,0,0.0,0.0,0.0,1.0,0.0,0.0,Ami matroi gaan shunlam 😊 Eta # CHAAMP best so...,ami matroi gaan shunlam 😊 eta chaamp best song...,17
1,Fear,tmi siccin series ta dekhio kmn voy er,5,0.0,0.0,1.0,0.0,0.0,0.0,tmi siccin series dekhio kmn voy,tmi siccin series dekhio kmn voy,6


In [ ]:
dataset_statistics(train_data)


Class Name:--> Joy 
Number of Words:16413
Number of Unique Words:5812
Maximum post length: 79
Average Words in texts: 12.041819515774028

Class Name:--> Sadness 
Number of Words:10177
Number of Unique Words:4182
Maximum post length: 53
Average Words in texts: 12.320823244552058

Class Name:--> Surprise 
Number of Words:2838
Number of Unique Words:1603
Maximum post length: 26
Average Words in texts: 12.393013100436681

Class Name:--> Disgust 
Number of Words:9574
Number of Unique Words:4036
Maximum post length: 43
Average Words in texts: 12.417639429312581

Class Name:--> Anger 
Number of Words:5621
Number of Unique Words:2761
Maximum post length: 77
Average Words in texts: 10.914563106796116

Class Name:--> Fear 
Number of Words:3254
Number of Unique Words:1506
Maximum post length: 23
Average Words in texts: 9.32378223495702


# Data Statistics

In [ ]:
df=train_data 
df['word_count'] = df.Text.apply(lambda x: len(str(x).split()))
more_than_150 =  df['word_count'] > 170
df3 = df[more_than_150]
df3

,Label,Text,enc_label,Anger,Disgust,Fear,Joy,Neutral,Sadness,Surprise,Texts,cleaned,word_count


In [ ]:
def char_length(row):
    return len(row.split())

In [ ]:
lables = ['Joy','Sadness','Surprise','Disgust','Anger','Fear','Neutral']

# Count number of unique words in each set
def dataset_statistics(dataset):
  for l in lables:
    word_list = [word.strip() for t in list(dataset[dataset[l]==1.0].Text) for word in t.strip().split()]
    print(f"\nClass Name:--> {l} ") 
    print("Number of Words:{}".format(len(word_list))) 
    print("Number of Unique Words:{}".format(len(np.unique(word_list)))) 
    print("Maximum post length: {}".format(max(dataset[dataset[l]==1.0].Text.apply(char_length))))
    print("Average Words in texts: {}".format(np.mean(dataset[dataset[l]==1.0].Text.apply(char_length))))
    

In [ ]:
train_data.head(2)

,Label,Text,enc_label,Anger,Disgust,Fear,Joy,Neutral,Sadness,Surprise,Texts,cleaned,word_count
0,Neutral,Live chate amke bolechilo rply debe,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Live chate amke bolechilo rply debe,live chate amke bolechilo rply debe,6
1,Disgust,"Maa , Maati , Manush ... Core ideologies of DI...",3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"Maa , Maati , Manush ... Core ideologies of DI...",maa maati manush core ideologies of didi getti...,22


In [ ]:
dataset_statistics(train_data)


Class Name:--> Joy 
Number of Words:24595
Number of Unique Words:5251
Maximum post length: 82
Average Words in texts: 17.283907238229094

Class Name:--> Sadness 
Number of Words:13312
Number of Unique Words:3162
Maximum post length: 48
Average Words in texts: 17.46981627296588

Class Name:--> Surprise 
Number of Words:4532
Number of Unique Words:1447
Maximum post length: 34
Average Words in texts: 17.91304347826087

Class Name:--> Disgust 
Number of Words:11470
Number of Unique Words:3199
Maximum post length: 63
Average Words in texts: 17.673343605546997

Class Name:--> Anger 
Number of Words:5266
Number of Unique Words:1679
Maximum post length: 36
Average Words in texts: 16.50783699059561

Class Name:--> Fear 
Number of Words:860
Number of Unique Words:413
Maximum post length: 28
Average Words in texts: 17.551020408163264

Class Name:--> Neutral 
Number of Words:10897
Number of Unique Words:3024
Maximum post length: 36
Average Words in texts: 14.433112582781456
